In [1]:
import pickle
# make sure you have the .pkl file because it doesn't work with the .zip
with open("sampled_SemCore+OMSTI.pkl", "rb") as f:
    data = pickle.load(f)
data_list = list(data.items())

# YOU ONLY NEED TO CHANGE THE WORD INDEX HERE 
target_word_index = 0

print(data_list[target_word_index][0])

# Structure of data_list
# data_list[word index][0-word, 1-all the info][meaning key][sentence No]['sentence'/'lematized'/'word'/'pos'/'id'/'sent_pos']
# 'sentence' = the whole sentence
# 'lematized' = lemmatization of words in sentence
# 'word' = target word
# 'pos' = NOUN/VERB/ADJ...
# 'id' = some id
# 'sent_pos' = position of target word in lemmatized sentence
keys = list(data_list[target_word_index][1].keys())

print(keys)

work
['work%2:35:02::', 'work%1:04:01::', 'work%2:41:00::']


In [ ]:
sentences_lem = []  # contains sentence, lemmatization, target word, ... in a dictionary
only_sentence = []  # contains only the sentences
only_lem = []       # contains lists of the lemmatized words in each sentence
for k in keys:
    for i in data_list[target_word_index][1][k]:
        sentences_lem.append(i)
        only_sentence.append(i['sentence'])
        only_lem.append(i['lemmatized'])

only_sentence

['The Fund works under the administration of the United Nations Development Programme and contributes in an innovative and comprehensive way to the reform process of the United Nations, in particular to the implementation of the One United Nations pilot programs.',
 'Bosnia and Herzegovina cannot work as a European State without Croats and Serbs, but they have too often been misguided and misled by leaders who put their own financial interests above the interests of their people.',
 '(f) Conducting public awareness and education campaigns to raise awareness about alternatives to imprisonment and how they work (resolution 1998/23, para. 3);',
 'At the same session, the Commission also highlighted overarching issues, including the need for research and development, capacity-building, technology transfer, information sharing and dissemination, mobilization of financial resources, making markets work effectively for sustainable development, and multi-stakeholder approaches and public parti

In [3]:
len(sentences_lem)

1305

In [4]:
only_lem[0]

[['the', 'DET'],
 ['Fund', 'NOUN'],
 ['work', 'VERB'],
 ['under', 'ADP'],
 ['the', 'DET'],
 ['administration', 'NOUN'],
 ['of', 'ADP'],
 ['the', 'DET'],
 ['United', 'NOUN'],
 ['Nations', 'NOUN'],
 ['Development', 'NOUN'],
 ['Programme', 'NOUN'],
 ['and', 'CONJ'],
 ['contribute', 'VERB'],
 ['in', 'ADP'],
 ['a', 'DET'],
 ['innovative', 'ADJ'],
 ['and', 'CONJ'],
 ['comprehensive', 'ADJ'],
 ['way', 'NOUN'],
 ['to', 'PRT'],
 ['the', 'DET'],
 ['reform', 'NOUN'],
 ['process', 'NOUN'],
 ['of', 'ADP'],
 ['the', 'DET'],
 ['United', 'NOUN'],
 ['Nations', 'NOUN'],
 [',', '.'],
 ['in', 'ADP'],
 ['particular', 'ADJ'],
 ['to', 'PRT'],
 ['the', 'DET'],
 ['implementation', 'NOUN'],
 ['of', 'ADP'],
 ['the', 'DET'],
 ['one', 'NUM'],
 ['United', 'NOUN'],
 ['Nations', 'NOUN'],
 ['pilot', 'NOUN'],
 ['program', 'NOUN'],
 ['.', '.']]

In [5]:
# lists of lists of the lemmatized words in the sentences
lem_words_in_sentences = []
for i in only_lem:
    sentence = []
    for j in i:
        sentence.append(j[0])
    lem_words_in_sentences.append(sentence)

lem_words_in_sentences

[['the',
  'Fund',
  'work',
  'under',
  'the',
  'administration',
  'of',
  'the',
  'United',
  'Nations',
  'Development',
  'Programme',
  'and',
  'contribute',
  'in',
  'a',
  'innovative',
  'and',
  'comprehensive',
  'way',
  'to',
  'the',
  'reform',
  'process',
  'of',
  'the',
  'United',
  'Nations',
  ',',
  'in',
  'particular',
  'to',
  'the',
  'implementation',
  'of',
  'the',
  'one',
  'United',
  'Nations',
  'pilot',
  'program',
  '.'],
 ['Bosnia',
  'and',
  'Herzegovina',
  'can',
  'not',
  'work',
  'as',
  'a',
  'european',
  'state',
  'without',
  'Croats',
  'and',
  'Serbs',
  ',',
  'but',
  'they',
  'have',
  'too',
  'often',
  'be',
  'misguided',
  'and',
  'mislead',
  'by',
  'leader',
  'who',
  'put',
  'they',
  'own',
  'financial',
  'interest',
  'above',
  'the',
  'interest',
  'of',
  'they',
  'people',
  '.'],
 ['(',
  'f',
  ')',
  'conduct',
  'public',
  'awareness',
  'and',
  'education',
  'campaign',
  'to',
  'raise',
 

In [6]:
# the original sentences but the words in them have been lemmatized
lemmatized_sentences = []
for l in lem_words_in_sentences:
    sen = ' '.join(l)
    print(sen)
    lemmatized_sentences.append(sen)

the Fund work under the administration of the United Nations Development Programme and contribute in a innovative and comprehensive way to the reform process of the United Nations , in particular to the implementation of the one United Nations pilot program .
Bosnia and Herzegovina can not work as a european state without Croats and Serbs , but they have too often be misguided and mislead by leader who put they own financial interest above the interest of they people .
( f ) conduct public awareness and education campaign to raise awareness about alternative to imprisonment and how they work ( resolution 1998/23 , para. 3 ) ;
at the same session , the Commission also highlight overarching issue , include the need for research and development , capacity-building , technology transfer , information sharing and dissemination , mobilization of financial resource , make market work effectively for sustainable development , and multi-stakeholder approach and public participation .
these mech

In [7]:
from transformers import BertTokenizer, BertModel
import torch

# Load the BERT-base-uncased model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)


c:\Users\vlad\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [8]:
# tokenize the list of sentences and get the hidden states
hidden_states = []

inputs = tokenizer(text=only_sentence, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

hidden_states = outputs.hidden_states

len(hidden_states)

13

In [9]:
print(only_sentence[0])
print(sentences_lem[0]['sent_pos'])
print(tokenizer(text=only_sentence[0], return_tensors="pt"))

print(only_sentence[4])
print(sentences_lem[4]['sent_pos'])
print(tokenizer(text=only_sentence[4], return_tensors="pt"))

print(only_sentence[5])
print(only_lem[5][sentences_lem[5]['sent_pos']])
print(tokenizer(text=only_sentence[5], return_tensors="pt"))

The Fund works under the administration of the United Nations Development Programme and contributes in an innovative and comprehensive way to the reform process of the United Nations, in particular to the implementation of the One United Nations pilot programs.
2
{'input_ids': tensor([[  101,  1996,  4636,  2573,  2104,  1996,  3447,  1997,  1996,  2142,
          3741,  2458,  4746,  1998, 16605,  1999,  2019,  9525,  1998,  7721,
          2126,  2000,  1996,  5290,  2832,  1997,  1996,  2142,  3741,  1010,
          1999,  3327,  2000,  1996,  7375,  1997,  1996,  2028,  2142,  3741,
          4405,  3454,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
These mechanisms are lik

In [10]:
target_word_hidden_states = []

for layer in range(13):
    tmp = []
    for i, sentence_data in enumerate(only_sentence):
        for form in ["work", "works", "working", "worked"]:
            try:
                target_word_position = list(tokenizer(text=sentence_data, return_tensors="pt")['input_ids'][0]).index(int(tokenizer(form, return_tensors="pt")['input_ids'][0][1]))
                break
            except:
                pass
        sentence_hidden_states = hidden_states[layer][i][target_word_position]
        tmp.append(sentence_hidden_states)

    target_word_hidden_states.append(tmp)

3
5
20
42
6
66
21
26
36
20
28
55
21
42
7
5
10
20
8
4
30
20
4
5
10
15
35
11
60
25
16
27
9
25
21
38
5
8
27
3
26
35
15
10
5
23
54
6
25
32
22
16
29
4
7
29
10
51
9
21
41
68
9
9
16
5
37
20
17
10
13
11
9
14
14
13
41
15
30
32
27
67
12
3
10
25
39
21
11
37
76
52
6
14
23
19
25
16
25
15
53
16
11
57
11
9
18
30
9
8
2
18
9
14
64
22
4
24
6
25
21
12
33
12
13
32
24
5
32
27
37
6
31
32
27
6
16
7
36
8
22
20
10
26
37
28
29
2
20
17
9
25
38
5
28
7
8
2
5
19
9
17
13
17
3
20
10
7
6
26
24
14
43
8
3
11
7
51
25
29
13
37
7
12
11
27
27
18
5
21
30
61
12
4
38
17
35
12
70
11
16
7
24
27
35
6
6
42
40
23
34
14
14
9
20
25
9
8
4
6
34
20
6
24
28
11
13
4
43
4
6
6
31
46
15
6
45
21
43
28
28
38
46
28
25
5
7
15
14
14
33
7
30
11
16
12
27
35
6
14
10
14
16
14
21
7
13
28
16
27
7
41
11
25
5
54
10
6
29
10
14
13
38
35
21
21
40
41
4
7
2
33
7
12
18
32
21
21
27
9
18
6
21
45
28
2
16
7
20
48
22
27
31
13
41
13
3
30
37
23
6
19
8
17
39
33
30
18
49
9
9
4
34
19
19
17
11
7
6
16
9
8
14
6
16
2
26
19
15
8
14
31
29
11
33
12
19
11
26
18
25
9
25
20
9
9
1

In [11]:
target_word_hidden_states[0][0]

tensor([ 6.1808e-01,  6.7678e-01, -2.1477e-01, -4.1657e-01,  5.1595e-01,
        -6.4280e-02,  6.0707e-01,  3.3547e-01, -5.6391e-01,  8.6561e-01,
         2.0325e+00,  2.7169e-01,  1.0314e+00, -1.1712e-01,  1.4025e-01,
        -2.4893e-01, -2.2516e-01,  1.4327e-01,  7.6488e-01, -1.2569e+00,
         2.8078e-01,  6.7350e-02, -4.1198e-01,  1.5716e-01,  3.0059e-01,
        -2.2133e-02, -1.8721e-01,  2.9267e-01, -4.5698e-01, -5.9825e-01,
         4.6466e-01,  8.7258e-01,  1.8903e-01,  4.5020e-01,  3.8778e-01,
        -1.3766e+00, -4.5839e-01,  7.3090e-01,  6.4887e-01,  6.9692e-01,
        -7.0778e-02,  3.2053e-01,  5.5517e-01, -8.7629e-02,  1.1021e-01,
         2.5666e-01,  1.1424e+00,  7.3349e-01,  1.8192e-01,  6.2673e-02,
         3.7669e-01,  1.2493e-01,  9.3404e-01,  1.4719e-01,  3.9751e-01,
        -8.9807e-01, -4.0741e-01,  1.1456e+00,  1.2595e+00,  1.1510e+00,
        -2.8457e-01,  8.8449e-02, -3.2919e-01,  3.3738e-01,  6.3108e-01,
         3.6132e-01, -6.2725e-01,  9.2032e-01,  7.1

In [12]:
def cosine_dist(v1, v2):
    dot_product = torch.dot(v1, v2)
    norm_v1 = torch.norm(v1)
    norm_v2 = torch.norm(v2)
    return 1 - dot_product / (norm_v1 * norm_v2)

signatures = []
for layer in range(1):
    signatures.append([])
    for i in target_word_hidden_states[layer]:
        signature = torch.tensor([])
        for j in target_word_hidden_states[layer]:
            signature = torch.cat((signature, cosine_dist(i, j).unsqueeze(0)))
        signatures[layer].append(signature)

In [13]:
signatures[0][0].shape

torch.Size([1305])

In [14]:
for layer in range(1):
    with open(f"layer_{layer}.in", "w") as f:
        for sig1 in signatures[layer]:
            for sig2 in signatures[layer]:
                f.write("{:.5f} ".format((1 / len(sig1)**0.5) * torch.norm(sig1 - sig2)))
            f.write("\n")